# Argparse + Decorators

In [1]:
def force(m, g):
    return m * g

In [2]:
f = force(50, 9.8)

print(f)

490.00000000000006


Как сделать так, чтоб можно было считать силу тяжести для разных значений массы и ускорения свободного падения?

## Not-the-Right Way

Вводить каждый раз нужные значения в исходный код программы.

In [3]:
if __name__ == '__main__':
    # f = force(50, 9.8)
    # f = force(70, 9.8)
    f = force(64, 9.7)
    
    print(f)

620.8


In [13]:
if __name__ == 'He-Who-Must-Not-Be-Named':
    print('Never printed')

## Input Way

Считать со входа.

In [5]:
if __name__ == '__main__':
    s = input()
    m, g = [float(n) for n in s.split()]
    
    f = force(m, g)
    
    print(f)

70 9.8
686.0


## Config Way

Считать из конфигурационного файла.

In [6]:
import json

In [16]:
config_data = {
    'mass': 65,
    'g': 9.8
}

with open('config.json', 'w') as f:
    f.write(json.dumps(config_data, indent=4))

In [17]:
! cat config.json

{
    "mass": 65,
    "g": 9.8
}

In [18]:
if __name__ == '__main__':
    data = json.loads(
        open('config.json').read()
    )
    
    f = force(m=data['mass'], g=data['g'])
    
    print(f)

637.0


Но даже если в конфиге указать все необходимые значения, то в коде всё равно нужно руками указывать путь до самого конфига...
Поэтому, если хотим запустить программу без ввода значений руками (`input`), то в любом случае приходим к передаче аргументов через командную строку.

## Command Line Args Way

In [21]:
import sys

In [22]:
if __name__ == '__main__':
    print(sys.argv)

['/home/alvant/lib/miniconda3/envs/nis/lib/python3.8/site-packages/ipykernel_launcher.py', '-f', '/home/alvant/.local/share/jupyter/runtime/kernel-e1c56099-a7d1-420e-9374-4ccd067c9ce4.json']


In [23]:
! cat src/args_intro.py

import sys


if __name__ == '__main__':
    print(sys.argv)


In [25]:
! python src/args_intro.py Hello world! 1 2 3

['src/args_intro.py', 'Hello', 'world!', '1', '2', '3']


In [26]:
! cat src/args_demo.py

import sys


def force(m, g):
    return m * g


def parse(args):
    m = float(args[0])
    g = float(args[1])

    return m, g


if __name__ == '__main__':
    m, g = parse(sys.argv[1:])
    f = force(m, g)

    print(f)


In [27]:
! python src/args_demo.py 60 9.8

588.0


In [29]:
# Ticket to the Moon

! python src/args_demo.py 50 1.6

80.0


## Command Line Args Way 2: Argparse

In [30]:
from argparse import ArgumentParser

In [31]:
parser = ArgumentParser()

parser.add_argument('m', type=float)
parser.add_argument('g', type=float)

_StoreAction(option_strings=[], dest='g', nargs=None, const=None, default=None, type=<class 'float'>, choices=None, help=None, metavar=None)

In [32]:
args = parser.parse_args(['12', '9.8'])  # sys.argv[1:]

In [33]:
args.m, args.g

(12.0, 9.8)

In [34]:
args = parser.parse_args(['9.8', '12'])

In [35]:
args.m, args.g

(9.8, 12.0)

In [36]:
parser = ArgumentParser()

parser.add_argument('-m')
parser.add_argument('-g')

_StoreAction(option_strings=['-g'], dest='g', nargs=None, const=None, default=None, type=None, choices=None, help=None, metavar=None)

In [37]:
args = parser.parse_args('-g 9.8 -m 12'.split())  # sys.argv[1:]

In [38]:
args.m, args.g

('12', '9.8')

In [39]:
parser = ArgumentParser()

parser.add_argument('-m', '--mass', type=float)
parser.add_argument('-g', type=float)

_StoreAction(option_strings=['-g'], dest='g', nargs=None, const=None, default=None, type=<class 'float'>, choices=None, help=None, metavar=None)

In [40]:
args = parser.parse_args(['-m', '12', '-g', '9.8'])  # sys.argv[1:]

In [41]:
args.mass, args.g

(12.0, 9.8)

In [42]:
! cat src/args_demo2.py

import sys
from argparse import ArgumentParser


def force(m, g):
    return m * g


def parse(args):

    return m, g


if __name__ == '__main__':
    parser = ArgumentParser()

    parser.add_argument('-m', '--mass', type=float)
    parser.add_argument('-g', type=float)

    args = parser.parse_args()
    
    f = force(args.mass, args.g)

    print(f)


In [43]:
! python src/args_demo2.py -m 50 -g 9.8

490.00000000000006


In [44]:
! python src/args_demo2.py -g 9.8 --mass 50

490.00000000000006


### Parsing Zoo

In [45]:
parser = ArgumentParser()

parser.add_argument('-m', '--mass', type=float)
parser.add_argument('-g', type=float, default=9.8)

args = parser.parse_args(['--mass', '60'])

print(f'Mass is {args.mass}, g is {args.g}.')

Mass is 60.0, g is 9.8.


In [46]:
parser = ArgumentParser()

parser.add_argument('--color', choices=['blue', 'green', 'brown'])

args = parser.parse_args("--color blue".split())

print(f"{args.color} eyes")

blue eyes


In [47]:
parser = ArgumentParser()

parser.add_argument(
    '--save',
    action='store_true',
    help="Whether to save something or not."
)

args = parser.parse_args("--save".split())

if args.save:
    print("Saving")

Saving


In [51]:
parser.parse_args("-h".split())

usage: ipykernel_launcher.py [-h] [--save]

optional arguments:
  -h, --help  show this help message and exit
  --save      Whether to save something or not.


SystemExit: 0

## Decorators

In [65]:
from numbers import Number
from typing import Callable


def add(num1: Number, num2: Number) -> Number:
    return num1 + num2

In [54]:
add(1, 2)

3

In [68]:
def add(*args) -> Number:
    print(args)
    
    return sum(args)

In [69]:
add(1, 2, 3)

(1, 2, 3)


6

In [83]:
def add(*args, **kwargs) -> Number:
    print(args, kwargs)
    
    if 'hello' in kwargs:
        print(f"Hello {kwargs['hello']}!")
    
    return sum(args)

In [84]:
add(1, 2, hello='world')

(1, 2) {'hello': 'world'}
Hello world!


3

In [72]:
def add(n1: Number, n2: Number) -> Number:
    return n1 + n2

def dummy_decorator(func: Callable) -> Callable:
    return add

In [73]:
def print_hello():
    print('Hello world!')

In [74]:
print_hello()

Hello world!


In [75]:
print_hello = dummy_decorator(print_hello)

In [76]:
print_hello(1, 2)

3

In [78]:
@dummy_decorator
def print_hello():
    print('Hello world!')

In [79]:
print_hello(1, 2)

3

In [81]:
def print_args(func: Callable) -> Callable:
    def new_func(*args, **kwargs):
        print(f'Args: {args}. Kwargs: {kwargs}.')
        
        return func(*args, **kwargs)
    
    return new_func


@print_args
def add(n1: Number, n2: Number) -> Number:
    return n1 + n2

In [82]:
add(1, 2)

Args: (1, 2). Kwargs: {}.


3

### Decorator Which Returns Decorator Which Returns Function Which Returns... 🤯

In [85]:
def world(greeted_object: str) -> Callable[[Callable], Callable]:
    def hello(func: Callable) -> Callable:
        def new_func(*args, **kwargs):
            print(f'Hello {greeted_object}!')
            
            return func(*args, **kwargs)
        
        return new_func
    
    return hello

In [86]:
@world('World')
def add(n1: Number, n2: Number) -> Number:
    return n1 + n2

In [87]:
add(0, -17.5)

Hello World!


-17.5

In [88]:
# Same as:

def add(n1: Number, n2: Number) -> Number:
    return n1 + n2

add = world('World')(add)

In [89]:
add(0, -17.5)

Hello World!


-17.5